In [1]:
import pandas as pd
import numpy as np
from functools import reduce
from operator import add
%matplotlib inline

pd.set_option('display.max_colwidth', 500)

In [2]:
train = pd.read_csv('../data/train.csv', index_col='Id')

In [3]:
seqs = {ix: pd.Series(x['Sequence'].split(',')) for ix, x in train.iterrows()}

# Size

In [4]:
train['Size'] = [len(seq) for seq in seqs.values()]

In [5]:
train[train.Size == 4][15:18]['Sequence']

Id
5487                             1,9,81,150094635296999121
5614    81647160420,170655787050,211212209880,227961624450
5698                                            1,2,8,7073
Name: Sequence, dtype: object

# ValuesSizeMean/Max/Min

In [6]:
train['ValuesSizeMean'] = [seq.apply(lambda x: len(x)).mean() for seq in seqs.values()]
train['ValuesSizeMax'] = [seq.apply(lambda x: len(x)).max() for seq in seqs.values()]
train['ValuesSizeMin'] = [seq.apply(lambda x: len(x)).min() for seq in seqs.values()]

In [7]:
train[train.ValuesSizeMax == 4][13:15]['Sequence']

Id
202             1,1,1,1,1,1,2,2,3,3,4,4,6,6,9,9,13,13,19,19,28,28,41,41,60,60,88,88,129,129,189,189,277,277,406,406,595,595,872,872,1278,1278,1873,1873,2745,2745,4023,4023,5896,5896,8641
205    3255,3257,3258,3259,3260,3261,3263,3264,3265,3266,3267,3269,3270,3271,3272,3273,3301,3302,3303,3304,3305,3307,3308,3309,3315,3316,3317,3319,3320,3321,3322,3323,3326,3327,3328,3329
Name: Sequence, dtype: object

# ValuesMode and ValuesModeFreq
ValuesMode is equals to the most frequent value, if it's the only

In [8]:
train['ValuesMode'] = [None if seqmode.size != 1 else seqmode[0] for seqmode in (seq.mode() for seq in seqs.values())]
train['ValuesModeFreq'] = [seq.value_counts().max() for seq in seqs.values()]
train.loc[train.ValuesMode.isnull(), 'ValuesModeFreq'] = None

In [9]:
train.loc[[2256], ['Sequence', 'ValuesMode', 'ValuesModeFreq']]

,Sequence,ValuesMode,ValuesModeFreq
Id,,,
2256,"0,0,35,0,3767,0",0,4


# LastValue

In [10]:
train['LastValue'] = [seq.iloc[-1] for seq in seqs.values()]

In [11]:
train.loc[[5698], ['Sequence', 'LastValue']]

,Sequence,LastValue
Id,,
5698,"1,2,8,7073",7073


# LastValue == ValuesMode

In [12]:
train['LastValueEqValuesMode'] = train.apply(lambda x: x['LastValue'] == x['ValuesMode'], axis=1)

In [14]:
train[train.LastValueEqValuesMode].loc[[2176], ['Sequence', 'LastValue']]

,Sequence,LastValue
Id,,
2176,"1,1,1,4,1,1,1,10,4,1,1,4,1,1,1,37,1,4,1,4,1,1,1,10,4,1,11,4,1,1,1",1


# NUniqueValues

In [15]:
train['NUniqueValues'] = [seq.value_counts().shape[0] for seq in seqs.values()]

# NPrefixValues
number of values that are prefix of next value

In [16]:
train['NPrefixValues'] = [reduce(add, (seq[i+1].startswith(seq[i]) for i in range(seq.shape[0] - 1)), 0) for seq in seqs.values()]

# NPrefixValues == Size - 1
pattern

x(0) = base

x(t+1) = concat(x(t), c(t+1))

In [17]:
train['NPrefixValuesEqSizeMinus1'] = (train.NPrefixValues == train.Size - 1)

In [18]:
train[(train.NPrefixValuesEqSizeMinus1) & (train.Size < 10) & (train.ValuesSizeMax < 5)][:2]

,Sequence,Size,ValuesSizeMean,ValuesSizeMax,ValuesSizeMin,ValuesMode,ValuesModeFreq,LastValue,LastValueEqValuesMode,NUniqueValues,NPrefixValues,NPrefixValuesEqSizeMinus1
Id,,,,,,,,,,,,
156118,"2,23,2357",3,2.333333,4,1,None,NaN,2357,False,3,2,True
191625,"2,21,212,2122",4,2.500000,4,1,None,NaN,2122,False,4,3,True


In [19]:
train[train.NPrefixValuesEqSizeMinus1].shape

(246, 12)

In [20]:
train.sample(2)

,Sequence,Size,ValuesSizeMean,ValuesSizeMax,ValuesSizeMin,ValuesMode,ValuesModeFreq,LastValue,LastValueEqValuesMode,NUniqueValues,NPrefixValues,NPrefixValuesEqSizeMinus1
Id,,,,,,,,,,,,
108461,"1,1,2,1,2,3,1,1,2,4,1,1,2,3,5,1,1,1,2,3,6,1,1,1,2,2,3,7,1,1,1,1,2,2,3,8,1,1,1,1,2,2,2,4,9,1,1,1,1,1,2,2,2,4,10",55,1.018182,2,1,1,26,10,False,10,23,False
94079,"9,18,27,36,45,54,63,72,82,83,84,85,86,87,88,89,90,99,108,117,126,135,144,153,163,164,165,166,167,168,169,170,171,180,189,198,207,216,225,234,244,245,246,247,248,249,250,251,252,261",50,2.620000,3,1,None,NaN,261,False,50,0,False
